# Notebook for automatic evaluation

### Generate the simplifications for GUTS:

In [4]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', '')))

Load the Generator 

In [1]:
from generator import Generator

generator = Generator()
# Load a model:
generator.reload("../models/GUTS.bin")
generator.eval()

ModuleNotFoundError: No module named 'generator'

#### Load evaluation data

In [2]:
import pandas as pd

tc_eval = pd.read_csv("../data/tc_eval.csv")
wiki_eval = pd.read_csv("../data/wiki_eval.csv")
wiki_eval.head()

,paragraph,size,guts,pivot
0,Erst ab 1860 unter Fürst Adolf Georg wurden di...,medium,Schon ab 1860 unter Fürst Ad wurden die Schlos...,Erst 1860 wurden die Palasträume komplett reno...
1,Im Jahr 2000 fand der österreichische Flieger ...,short,Im Jahr 2000 fand die österreichische Flieger ...,Im Jahr 2000 fand der österreichische Pilot Si...
2,Der Ort Arbing ist am Fuß der mittelalterliche...,long,Der Ort Arbing am Fuß der mittelalterlichen St...,Das Dorf Arbing liegt am Fuße des mittelalterl...
3,Die Ortschaften im Verwaltungsbezirk der Bürge...,long,Die Ortschaften im Verwaltungsgebiet der Bürge...,"Vor 1794 gehörte Bleialf zu Prüm, dessen Fürst..."
4,"Die Ortschaft ist über die A726, die Strathave...",medium,"Die Ortschaft ist über A7, die Strathaven mit ...",Das Dorf ist mit dem Straßennetz über die A726...


Simplify the TextComplexityDE data with GUTS and append them to the dataframe

In [ ]:
# Simplify TC data with GUTS 

model_output = []

for idx, row in tc_eval.iterrows():
    p = row['paragraph']
    greedy_simplification = generator.run(p, sample_size=1, top_k=1, top_p=0)[0]
    model_output.append(greedy_simplification)

    tc_eval['guts'] = model_output
    
tc_eval.to_csv("../data/tc_eval.csv", index=False)

Simplify the Wikipedia data with GUTS and append them to the dataframe

In [ ]:
# Simplify Wiki data with GUTS

model_output = []

for idx, row in wiki_eval.iterrows():
    p = row['paragraph']
    greedy_simplification = generator.run(p, sample_size=1, top_k=1, top_p=0)[0]
    model_output.append(greedy_simplification)

    wiki_eval['guts'] = model_output

wiki_eval.to_csv("../data/wiki_eval.csv", index=False)

^^Testing different decoding methods:

In [9]:
t = "Die ist ein kleiner Test. Der Text ist so einfach, dass es leichter ist diesen komplexer zu machen. Naja, mal sehen was GUTS daraus macht!"
#print(generator.run(t, top_k=1, top_p=1)[0]) # Greedy decoding
#print(generator.run(t, top_k=5, top_p=1)[0]) # Top-K sampling (K=5)
#print(generator.run(t, top_k=0, top_p=0.95)[0]) # Nucleus sampling
print(generator.run_typical(t, sample_size=1, p=0.9)[0]) # Typical sampling

Nach dem kleinen Test ist der Text so einfach, das sind die komplexer zu machen und macht. Naja, was es kann seine Arbeit erleichtern und helfen ihm dabei, den Alltag zu meistern. Der Jungs haben Spaß dabei, ihre Arbeiten fertig zu stellen. Dabei kann man sich sehr gut mit den vielen Arbeitsschritte beschäftigen und ist sehr motiviert den neuen Weg des Mannes zu ebnen.



## Automatic Evaluation

### Prepare the evaluation metrics

In [5]:
import textstat
import nltk.translate.bleu_score as bleu
import numpy as np
import pandas as pd

from misc.utils_evaluation import SARIsent
from reward.simplicity import LexicalSimplicity, SyntacticSimplicity
from misc.utils_text import LemmaModel
from reward.meaning_preservation import TextSimilarity
from bert_score import BERTScorer

bert_scorer = BERTScorer("dbmdz/bert-base-german-cased", num_layers=5)
text_sim = TextSimilarity(bert_scorer)

textstat.set_lang('de')

lemma_model = LemmaModel(model_path=os.path.abspath(os.path.join('..', './models/morphmodel_ger.pgz')))
lex_simpl = LexicalSimplicity(lemma_model)
syntactic_simpl = SyntacticSimplicity()

# FRE 
def get_fre(txt):
    return textstat.flesch_reading_ease(txt)

# BLEU 
def get_sent_bleu(refs, cand):
    if type(refs) == str:
        refs = [refs]
    refs = [r.split() for r in refs]
    cand = cand.split()
    return bleu.sentence_bleu(refs, cand)

def get_corpus_bleu(ref, cand):
    ref = ref.split()
    ref = [[ref]]
    cand = [cand.split()]
    return bleu.corpus_bleu(ref, cand)

def compute_compression_ratio(source, generated):
    return float(len(generated) / len(source))

def score(source, output, reference):
    # Fix bad inputs:
    if type(output) != str:
        output = ''

    # Check if reference simplification exists
    if reference != None: 
        bleu = get_corpus_bleu(reference, output)
        sari = SARIsent(source, output, [reference])
    else:
        bleu = 0
        sari = 0

    fre_value = get_fre(output)
    fre_improvement = fre_value - get_fre(source)

    zipf_improv = np.mean(lex_simpl.zipf_score(output)) - np.mean(lex_simpl.zipf_score(source))
    if zipf_improv == np.nan:
        zipf_improv = 0

    compression = compute_compression_ratio(source, output)

    coverage = text_sim.score([source], [output])[0]

    return {"sari":sari, "bleu":bleu, "fre_value": fre_value, "fre_improv":fre_improvement, "zipf_improv":zipf_improv, "compression": compression, 'coverage': coverage}

c:\Users\ba3ev2\Work Folders\Documents\repos\GUTS\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 59.0/59.0 [00:00<?, ?B/s]
Downloading: 100%|██████████| 456/456 [00:00<00:00, 434kB/s]
Downloading: 100%|██████████| 234k/234k [00:00<00:00, 8.85MB/s]
Downloading:  99%|█████████▉| 419M/422M [00:25<00:00, 17.4MB/s] 

KeyboardInterrupt: 

Downloading:  99%|█████████▉| 419M/422M [00:37<00:00, 17.4MB/s]

### Evaluation of the data

Load the evaluation data

In [9]:
import pandas as pd

tc_eval = pd.read_csv("../data/tc_eval.csv")
tc_eval.head()

,paragraph,reference,pivot,guts1,guts2
0,"Wegen dieser leichten Vergänglichkeit wurde ,S...","Weil Seifenblasen nicht lange halten, wurden s...",Aufgrund dieser leichten Vergänglichkeit wurde...,"Das leichte Vergänglichkeit wurde ,Das Blase‘ ...","Mit dieser leichten Vergänglichkeit wird ,Seif..."
1,"Eine Seifenblase entsteht, wenn sich ein dünne...","Eine Seifenblase entsteht, wenn sich eine klei...","Eine Seifenblase entsteht, wenn ein dünner Was...","Die Seifenblase entsteht, sobald sich ein dünn...","Ein Seifenblase entsteht,, wenn sich ein dicke..."
2,Die Schichtdicke der Seifenblase lässt sich be...,Die Dicke der Seifenblase lässt sich beobachte...,Die Schichtdicke der Seifenblase kann beobacht...,Die Schichtdicke ist die Seifenblase lässt sic...,Die Schichtdicke des Seifenblasen lässt sich b...
3,"So schildert der achte Gesang beispielsweise, ...",So schildert der achte Besang beispielsweise f...,"Zum Beispiel beschreibt das achte Lied, wie er...",Der achte Gesang schildert den achte Gesang be...,"Der achte Gesang hingegen, wie im Haus des Pha..."
4,Vergleichbare Stellen belegen eine sorgfältige...,"Es gibt Schriften, die erzählen, dass es üblic...",Ähnliche Orte zeigen eine sorgfältige Reinigun...,Esbare Stellen belegen eine gründliche Reinigu...,Berbare Stellen belegen eine gründliche Reinig...


In [10]:
wiki_eval = pd.read_csv("../data/wiki_eval.csv")
wiki_eval.head()

,paragraph,size,guts1,guts2,pivot
0,Erst ab 1860 unter Fürst Adolf Georg wurden di...,medium,Erst ab 1860 unter Fürsten Adolf Georg wurden ...,Schon ab 1860 unter Fürst Ad wurden die Schlos...,Erst 1860 wurden die Palasträume komplett reno...
1,Im Jahr 2000 fand der österreichische Flieger ...,short,Das Jahr 2000 fand der österreichischen Fliege...,Im Jahr 2000 fand die österreichische Flieger ...,Im Jahr 2000 fand der österreichische Pilot Si...
2,Der Ort Arbing ist am Fuß der mittelalterliche...,long,Das Ort Arbing ist im 15 Gebäude entstanden un...,Der Ort Arbing am Fuß der mittelalterlichen St...,Das Dorf Arbing liegt am Fuße des mittelalterl...
3,Die Ortschaften im Verwaltungsbezirk der Bürge...,long,Die Ortschaften im Verwaltungs Bezirk der Bürg...,Die Ortschaften im Verwaltungsgebiet der Bürge...,"Vor 1794 gehörte Bleialf zu Prüm, dessen Fürst..."
4,"Die Ortschaft ist über die A726, die Strathave...",medium,"Die Ortschaft ist über den A726, über East Kil...","Die Ortschaft ist über A7, die Strathaven mit ...",Das Dorf ist mit dem Straßennetz über die A726...


TextComplexity evaluation

In [11]:
def print_tceval_result(df, simpl_column):
    bleus = []
    saris = []
    fre_values = []
    fre_improvs = []
    zipf_improvs = []
    compressions = []
    coverages = []

    for idx, row in df.iterrows():
        p = row['paragraph']
        ref = row['reference']
        simpl = row[simpl_column]

        scores = score(p, simpl, ref)

        bleus.append(scores['bleu'])
        saris.append(scores['sari'])
        fre_values.append(scores['fre_value'])
        fre_improvs.append(scores['fre_improv'])
        zipf_improvs.append(scores['zipf_improv'])
        compressions.append(scores['compression'])
        coverages.append(scores['coverage'])

    print("%s: " % simpl_column)
    print("BLEU: ", np.mean(bleus))
    print("SARI: ", np.mean(saris))
    print("FRE Values: ", np.mean(fre_values))
    print("FRE Improvements: ", np.mean(fre_improvs))
    print("Zipf Improvements: ", np.mean(zipf_improvs))
    print("Compression: ", np.mean(compressions))
    print("Coverages: ", np.mean(coverages))

In [12]:
print_tceval_result(tc_eval, "guts")

C:\Python37\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
Batches: 100%|██████████| 1/1 [00:00<00:00, 124.98it/s]
C:\Python37\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
Batches: 100%|██████████| 1/1 [00:00<00:00, 90.91it/s]
C:\Python37\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contain

guts1: 
BLEU:  0.04503604718096531
SARI:  0.3476000715415094
FRE Values:  41.370879120879124
FRE Improvements:  15.718021978021977
Zipf Improvements:  0.23120466903367873
Compression:  0.8027637537085042
Coverages:  0.9375636710008984


In [13]:
print_tceval_result(tc_eval, "guts2")

Batches: 100%|██████████| 1/1 [00:00<00:00, 99.97it/s]

guts2: 
BLEU:  0.04496926996862391
SARI:  0.3478695542835336
FRE Values:  37.448241758241764
FRE Improvements:  11.795384615384616
Zipf Improvements:  0.058724208195551235
Compression:  0.788778436644369
Coverages:  0.8754736081009677


In [14]:
print_tceval_result(tc_eval, "pivot")

Batches: 100%|██████████| 1/1 [00:00<00:00, 99.98it/s]

pivot: 
BLEU:  0.0636725315460017
SARI:  0.37028442739737477
FRE Values:  38.7123076923077
FRE Improvements:  13.05945054945055
Zipf Improvements:  0.20646264588191046
Compression:  0.8626900024554793
Coverages:  0.7273270703956329


Wikipedia paragraph evaluation

In [17]:
def print_wikieval_result(df, simpl_column):
    fre_values = []
    fre_improvs = []
    zipf_improvs = []
    compressions = []
    coverages = []

    for idx, row in df.iterrows():
        p = row['paragraph']
        simpl = row[simpl_column]

        scores = score(p, simpl, None)
        fre_values.append(scores['fre_value'])
        fre_improvs.append(scores['fre_improv'])
        zipf_improvs.append(scores['zipf_improv'])
        compressions.append(scores['compression'])
        coverages.append(scores['coverage'])

    print("%s: " % simpl_column)
    print("FRE Values: ", np.mean(fre_values))
    print("FRE Improvements: ", np.mean(fre_improvs))
    print("Zipf Improvements: ", np.mean(zipf_improvs))
    print("Compression: ", np.mean(compressions))
    print("Coverages: ", np.mean(coverages))

In [32]:
print_wikieval_result(wiki_eval, "guts")

Batches: 100%|██████████| 1/1 [00:00<00:00, 90.91it/s]
C:\Python37\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Python37\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Batches: 100%|██████████| 1/1 [00:00<00:00, 76.92it/s]


guts2: 
FRE Values:  53.5533
FRE Improvements:  9.768666666666666
Zipf Improvements:  nan
Compression:  0.7290298253428983
Coverages:  0.815825131531706


In [20]:
print_wikieval_result(wiki_eval, "pivot")

Batches: 100%|██████████| 1/1 [00:00<00:00, 100.01it/s]


pivot: 
FRE Values:  50.187133333333335
FRE Improvements:  6.4025
Zipf Improvements:  0.24343451477062023
Compression:  0.7660105432800238
Coverages:  0.5494037926192546
